In [1]:
!pip install ortools
!pip install xlsxwriter
!pip install openpyxl

In [8]:
import os
import pandas as pd
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt

In [9]:
# =====================
# 1. 데이터 로드
# =====================
df = pd.read_csv("학급반편성CSP 문제 입력파일.csv", encoding="utf-8-sig")

# ID를 int로 고정
df["id"] = df["id"].astype(int)
students = df["id"].tolist()
num_students = len(students)

In [10]:
# 클래스 수 및 크기
num_classes = 6
class_sizes = [34, 34, 33, 33, 33, 33]  # 총 200명

# 속성 딕셔너리
scores = df.set_index("id")["score"].to_dict()
genders = df.set_index("id")["sex"].to_dict()
last_class = df.set_index("id")["24년 학급"].to_dict()
clubs = df.set_index("id")["클럽"].to_dict()

# Boolean 속성
leaders = df.set_index("id")["Leadership"].apply(lambda x: 1 if str(x).lower()=="yes" else 0).to_dict()
piano = df.set_index("id")["Piano"].apply(lambda x: 1 if str(x).lower()=="yes" else 0).to_dict()
non_attend = df.set_index("id")["비등교"].apply(lambda x: 1 if str(x).lower()=="yes" else 0).to_dict()
sports = df.set_index("id")["운동선호"].apply(lambda x: 1 if str(x).lower()=="yes" else 0).to_dict()

# 좋은관계
care_pairs = []
for _, row in df.iterrows():
    if pd.notna(row["좋은관계"]):
        s1 = int(row["id"])
        s2 = int(row["좋은관계"])
        care_pairs.append((s1, s2))

# 나쁜관계
dislike_pairs = []
for _, row in df.iterrows():
    if pd.notna(row["나쁜관계"]):
        s1 = int(row["id"])
        s2 = int(row["나쁜관계"])
        dislike_pairs.append((s1, s2))

In [11]:
# =====================
# 2. CP-SAT 모델 정의
# =====================
model = cp_model.CpModel()

# 각 학생이 어느 반에 속하는지 (정수 변수)
student_vars = {s: model.NewIntVar(0, num_classes-1, f"student_{s}") for s in students}

# 반별 배정 표시 변수
assigned = {
    s: [model.NewBoolVar(f"assigned_{s}_{c}") for c in range(num_classes)]
    for s in students
}

for s in students:
    model.AddAllowedAssignments([student_vars[s]], [[c] for c in range(num_classes)])
    for c in range(num_classes):
        model.Add(student_vars[s] == c).OnlyEnforceIf(assigned[s][c])
        model.Add(student_vars[s] != c).OnlyEnforceIf(assigned[s][c].Not())

In [12]:
# =====================
# 3. 제약조건
# =====================

# (1) 반별 인원 정확히 맞추기
for c, size in enumerate(class_sizes):
    model.Add(sum(assigned[s][c] for s in students) == size)

# (2) 불화 학생은 같은 반 금지
for s1, s2 in dislike_pairs:
    model.Add(student_vars[s1] != student_vars[s2])

# (3) 비등교 학생 보호 (비등교 학생만 좋은 관계 유지)
for s1, s2 in care_pairs:
    if non_attend[s1] == 1:
        model.Add(student_vars[s1] == student_vars[s2])

# (4) 각 반 최소 1명 리더
for c in range(num_classes):
    model.Add(sum(assigned[s][c] * leaders[s] for s in students) >= 1)

# (5) 피아노 균등 분배
total_piano = sum(piano.values())
target_piano = total_piano // num_classes
for c in range(num_classes):
    model.Add(sum(assigned[s][c] * piano[s] for s in students) >= target_piano - 1)
    model.Add(sum(assigned[s][c] * piano[s] for s in students) <= target_piano + 1)

# (6) 성적 분배 균형
total_score = sum(scores.values())
class_scores = []
for c in range(num_classes):
    class_score = model.NewIntVar(0, total_score, f"class_{c}_score")
    model.Add(class_score == sum(assigned[s][c] * scores[s] for s in students))
    class_scores.append(class_score)

max_score = model.NewIntVar(0, total_score, "max_score")
min_score = model.NewIntVar(0, total_score, "min_score")
model.AddMaxEquality(max_score, class_scores)
model.AddMinEquality(min_score, class_scores)

# (7) 비등교 학생 균등 분배
total_non = sum(non_attend.values())
target_non = max(1, total_non // num_classes)
for c in range(num_classes):
    model.Add(sum(assigned[s][c] * non_attend[s] for s in students) >= target_non - 1)
    model.Add(sum(assigned[s][c] * non_attend[s] for s in students) <= target_non + 1)

# (8) 남녀 비율 균등
male_total = sum(1 for g in genders.values() if g == "boy")
target_male = male_total // num_classes
for c in range(num_classes):
    model.Add(sum(assigned[s][c] * (1 if genders[s] == "boy" else 0) for s in students) >= target_male - 2)
    model.Add(sum(assigned[s][c] * (1 if genders[s] == "boy" else 0) for s in students) <= target_male + 2)

# (9) 운동 능력 균등
total_sports = sum(sports.values())
target_sports = total_sports // num_classes
for c in range(num_classes):
    model.Add(sum(assigned[s][c] * sports[s] for s in students) >= target_sports - 1)
    model.Add(sum(assigned[s][c] * sports[s] for s in students) <= target_sports + 1)

# (10) 전년도 같은 반 최소화 (soft constraint)
same_class_indicators = []
for i in range(len(students)):
    for j in range(i+1, len(students)):
        if last_class[students[i]] == last_class[students[j]]:
            b = model.NewBoolVar(f"same_last_{students[i]}_{students[j]}")
            model.Add(student_vars[students[i]] == student_vars[students[j]]).OnlyEnforceIf(b)
            model.Add(student_vars[students[i]] != student_vars[students[j]]).OnlyEnforceIf(b.Not())
            same_class_indicators.append(b)

if same_class_indicators:
    penalty_same_last = model.NewIntVar(0, len(same_class_indicators), "penalty_same_last")
    model.Add(penalty_same_last == sum(same_class_indicators))
else:
    penalty_same_last = model.NewIntVar(0, 0, "penalty_same_last")

# (11) 클럽 활동 균등 분배
club_names = set(clubs.values())
for club in club_names:
    if pd.isna(club):  # 클럽 없는 학생 제외
        continue
    club_members = [s for s in students if clubs[s] == club]
    total_club = len(club_members)
    target_club = total_club // num_classes
    for c in range(num_classes):
        model.Add(sum(assigned[s][c] for s in club_members) >= target_club - 1)
        model.Add(sum(assigned[s][c] for s in club_members) <= target_club + 1)


In [13]:
# =====================
# 4. Solver 실행
# =====================
solver = cp_model.CpSolver()
status = solver.Solve(model)

# =====================
# 5. 결과 저장 및 출력
# =====================
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    results = []
    for s in students:
        assigned_class = solver.Value(student_vars[s]) + 1
        row = df[df["id"] == s].copy()
        row.insert(1, "assigned_class", assigned_class)
        results.append(row)

    result_df = pd.concat(results, ignore_index=True)

    # 터미널 출력
    for c in range(1, num_classes+1):
        class_df = result_df[result_df["assigned_class"] == c]
        avg_score = round(class_df["score"].mean(), 2)
        print(f"--- Class {c} ---")
        print("인원:", len(class_df), ", 평균 성적:", avg_score)
        print("학생 목록:", class_df["id"].tolist())
        print()

    # 엑셀 저장
    save_path = "class_assignment_result.xlsx"
    with pd.ExcelWriter(save_path, engine="xlsxwriter") as writer:
        # 전체결과
        result_df.to_excel(writer, sheet_name="전체결과", index=False)
        # 반별 시트
        for c in range(1, num_classes+1):
            class_df = result_df[result_df["assigned_class"] == c]
            class_df.to_excel(writer, sheet_name=f"반{c}", index=False)

        # 요약 통계
        summary_data = []
        for c in range(1, num_classes+1):
            class_df = result_df[result_df["assigned_class"] == c]
            avg_score = class_df["score"].mean()
            male_count = (class_df["sex"] == "boy").sum()
            female_count = (class_df["sex"] == "girl").sum()
            non_count = (class_df["비등교"].str.lower() == "yes").sum()
            leader_count = (class_df["Leadership"].str.lower() == "yes").sum()
            piano_count = (class_df["Piano"].str.lower() == "yes").sum()
            sports_count = (class_df["운동선호"].str.lower() == "yes").sum()
            summary_data.append({
                "반": c,
                "학생 수": len(class_df),
                "평균 성적": round(avg_score, 2),
                "남학생 수": male_count,
                "여학생 수": female_count,
                "남녀 비율": f"{male_count}:{female_count}",
                "비등교 학생 수": non_count,
                "리더십 학생 수": leader_count,
                "피아노 학생 수": piano_count,
                "운동선호 학생 수": sports_count
            })
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name="요약통계", index=False)

        # 교우관계
        relation_results = []
        for s1, s2 in care_pairs:
            class1 = solver.Value(student_vars[s1]) + 1
            class2 = solver.Value(student_vars[s2]) + 1
            relation_results.append({
                "학생": s1,
                "상대": s2,
                "관계": "좋은관계",
                "학생 반": class1,
                "상대 반": class2
            })
        for s1, s2 in dislike_pairs:
            class1 = solver.Value(student_vars[s1]) + 1
            class2 = solver.Value(student_vars[s2]) + 1
            relation_results.append({
                "학생": s1,
                "상대": s2,
                "관계": "나쁜관계",
                "학생 반": class1,
                "상대 반": class2
            })
        relation_df = pd.DataFrame(relation_results)
        relation_df.to_excel(writer, sheet_name="교우관계", index=False)

        # 클럽 분포 (원형 차트 이미지 저장 후 경로 삽입)
        workbook  = writer.book
        for c in range(1, num_classes+1):
            class_df = result_df[result_df["assigned_class"] == c]
            club_counts = class_df["클럽"].value_counts()

            fig, ax = plt.subplots()
            ax.pie(club_counts, labels=club_counts.index, autopct="%1.1f%%")
            ax.set_title(f"{c}반 클럽 분포")
            img_path = f"club_plot_{c}.png"
            plt.savefig(img_path, bbox_inches="tight")
            plt.close(fig)

            worksheet = workbook.add_worksheet(f"클럽분포{c}")
            worksheet.insert_image("B2", img_path)

    print(f"✅ 결과가 {save_path}에 저장되었습니다.")
else:
    print("❌ Solver가 해를 찾지 못했습니다.")

--- Class 1 ---
인원: 34 , 평균 성적: 76.29
학생 목록: [202510, 202535, 202536, 202544, 202545, 202551, 202552, 202553, 202555, 202556, 202558, 202559, 202564, 202568, 202573, 202575, 202580, 202583, 202589, 202594, 202599, 202610, 202625, 202627, 202630, 202635, 202644, 202648, 202649, 202654, 202659, 202679, 202692, 202698]

--- Class 2 ---
인원: 34 , 평균 성적: 79.38
학생 목록: [202505, 202506, 202516, 202518, 202520, 202524, 202527, 202528, 202530, 202541, 202548, 202554, 202561, 202576, 202577, 202590, 202598, 202601, 202605, 202606, 202617, 202618, 202620, 202623, 202637, 202642, 202656, 202658, 202660, 202664, 202666, 202686, 202688, 202694]

--- Class 3 ---
인원: 33 , 평균 성적: 82.18
학생 목록: [202503, 202508, 202509, 202522, 202529, 202538, 202540, 202562, 202563, 202572, 202579, 202585, 202586, 202595, 202597, 202603, 202604, 202608, 202622, 202631, 202634, 202641, 202652, 202662, 202669, 202673, 202675, 202677, 202680, 202681, 202682, 202690, 202691]

--- Class 4 ---
인원: 33 , 평균 성적: 79.85
학생 목록: [20250

/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 48152 (\N{HANGUL SYLLABLE BAN}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 53364 (\N{HANGUL SYLLABLE KEUL}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 47101 (\N{HANGUL SYLLABLE REOB}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 48516 (\N{HANGUL SYLLABLE BUN}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 54252 (\N{HANGUL SYLLABLE PO}) missing 

✅ 결과가 class_assignment_result.xlsx에 저장되었습니다.


/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 48152 (\N{HANGUL SYLLABLE BAN}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 53364 (\N{HANGUL SYLLABLE KEUL}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 47101 (\N{HANGUL SYLLABLE REOB}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 48516 (\N{HANGUL SYLLABLE BUN}) missing from font(s) DejaVu Sans.
  plt.savefig(img_path, bbox_inches="tight")
/var/folders/87/dfpp2xds1sj0q6xgdk06s6xr0000gn/T/ipykernel_30743/2639327170.py:100: UserWarning: Glyph 54252 (\N{HANGUL SYLLABLE PO}) missing 

In [15]:
import os
import pandas as pd


plt.rc("font", family="AppleGothic") 

save_dir = os.path.expanduser("~/ai_planning")
os.makedirs(save_dir, exist_ok=True)

excel_path = os.path.join(save_dir, "class_assignment_result.xlsx")


with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
    # --------------------------
    # 1. 전체결과 시트
    # --------------------------
    result_df.to_excel(writer, sheet_name="전체결과", index=False)
    
    # --------------------------
    # 2. 반별 시트 (반1~반6)
    # --------------------------
    for c in range(1, num_classes+1):
        class_df = result_df[result_df["assigned_class"] == c]
        class_df.to_excel(writer, sheet_name=f"반{c}", index=False)
    
    # --------------------------
    # 3. 요약 통계 시트
    # --------------------------
    summary_data = []
    for c in range(1, num_classes+1):
        class_df = result_df[result_df["assigned_class"] == c]
        avg_score = class_df["score"].mean()
        male_count = (class_df["sex"] == "boy").sum()
        female_count = (class_df["sex"] == "girl").sum()
        non_count = (class_df["비등교"].str.lower() == "yes").sum()
        leader_count = (class_df["Leadership"].str.lower() == "yes").sum()
        piano_count = (class_df["Piano"].str.lower() == "yes").sum()
        sports_count = (class_df["운동선호"].str.lower() == "yes").sum()
        
        summary_data.append({
            "반": c,
            "학생 수": len(class_df),
            "평균 성적": round(avg_score, 2),
            "남학생 수": male_count,
            "여학생 수": female_count,
            "남녀 비율": f"{male_count}:{female_count}",
            "비등교 학생 수": non_count,
            "리더십 학생 수": leader_count,
            "피아노 학생 수": piano_count,
            "운동선호 학생 수": sports_count
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name="요약통계", index=False)
    
    # --------------------------
    # 4. 교우 관계 시트 (반 번호 + 같은 반 여부)
    # --------------------------
    relation_results = []

    # 좋은 관계
    for s1, s2 in care_pairs:
        class1 = solver.Value(student_vars[s1]) + 1
        class2 = solver.Value(student_vars[s2]) + 1
        relation_results.append({
            "학생": s1,
            "상대": s2,
            "관계": "좋은관계",
            "학생 반": class1,
            "상대 반": class2,
            "같은 반 여부": "같은 반" if class1 == class2 else "다른 반"
        })
    
    # 나쁜 관계
    for s1, s2 in dislike_pairs:
        class1 = solver.Value(student_vars[s1]) + 1
        class2 = solver.Value(student_vars[s2]) + 1
        relation_results.append({
            "학생": s1,
            "상대": s2,
            "관계": "나쁜관계",
            "학생 반": class1,
            "상대 반": class2,
            "같은 반 여부": "⚠️ 같은 반(문제)" if class1 == class2 else "다른 반"
        })
    
    relation_df = pd.DataFrame(relation_results)
    relation_df.to_excel(writer, sheet_name="교우관계", index=False)

    # --------------------------
    # 5. 클럽 분포 시각화 시트 (원형차트)
    # --------------------------
    worksheet = writer.book.add_worksheet("클럽분포")
    writer.sheets["클럽분포"] = worksheet
    
    row_offset = 0
    for c in range(1, num_classes+1):
        class_df = result_df[result_df["assigned_class"] == c]
        club_counts = class_df["클럽"].value_counts()
    
        # DataFrame 저장
        club_df = club_counts.reset_index()
        club_df.columns = ["클럽", "인원수"]
        club_df.to_excel(writer, sheet_name="클럽분포", startrow=row_offset, index=False)
    
        # 시각화 (파이차트)
        plt.figure(figsize=(5,5))
        plt.pie(club_counts, labels=club_counts.index, autopct='%1.1f%%', startangle=90)
        plt.title(f"{c}반 클럽 분포")
        plt.tight_layout()
    
        img_path = f"club_plot_{c}.png"
        plt.savefig(img_path)
        plt.close()
    
        # 엑셀에 이미지 삽입
        worksheet.insert_image(row_offset, 4, img_path)
    
        row_offset += len(club_df) + 20

print(f"✅ 배정 결과 + 요약 통계 + 교우 관계가 {excel_path} 에 저장")


✅ 배정 결과 + 요약 통계 + 교우 관계가 /Users/suuu/ai_planning/class_assignment_result.xlsx 에 저장
